In [1]:
import pandas as pd
import numpy as np
import os
import requests
from time import sleep

pd.set_option('display.max_columns', None)

# Scryfall API

In [3]:
cards_url = 'https://api.scryfall.com/cards?page=1'

In [4]:
res = requests.get(cards_url)

In [5]:
len(res.json()['data'])

175

In [9]:
BATCH_LIMIT = 25
CARDS_PER_PAGE = 175

In [11]:
dynamo_batch = [(first, first+BATCH_LIMIT) for first in list(range(0, CARDS_PER_PAGE, BATCH_LIMIT))]
dynamo_batch

[(0, 25), (25, 50), (50, 75), (75, 100), (100, 125), (125, 150), (150, 175)]

In [21]:
batch = dynamo_batch[0]
batch

(0, 25)

In [34]:
list(range(5, 5))

[]

In [23]:
table = 'global-cards-table'

In [142]:
cards = res.json()['data'][batch[0]:batch[1]]

In [140]:
len(cards[0]['mana_cost'])

0

In [143]:
for idx, card in enumerate(cards):
    for key, val in card.items():
        cards[idx][key] = batch_format(val)
    cards[idx]['cardId'] = cards[idx]['id']

In [144]:
cards

[{'object': {'S': 'card'},
  'id': {'S': '61dc9a80-850c-4945-b84a-85381cc3ee85'},
  'oracle_id': {'S': 'e521322b-0e83-458c-8936-7021a80ee279'},
  'multiverse_ids': {'SS': []},
  'tcgplayer_id': {'N': '206636'},
  'name': {'S': 'Temple of Plenty'},
  'lang': {'S': 'en'},
  'released_at': {'S': '2020-01-24'},
  'uri': {'S': 'https://api.scryfall.com/cards/61dc9a80-850c-4945-b84a-85381cc3ee85'},
  'scryfall_uri': {'S': 'https://scryfall.com/card/thb/351/temple-of-plenty?utm_source=api'},
  'layout': {'S': 'normal'},
  'highres_image': {'BOOL': True},
  'image_uris': {'M': {'small': {'S': 'https://img.scryfall.com/cards/small/front/6/1/61dc9a80-850c-4945-b84a-85381cc3ee85.jpg?1577717404'},
    'normal': {'S': 'https://img.scryfall.com/cards/normal/front/6/1/61dc9a80-850c-4945-b84a-85381cc3ee85.jpg?1577717404'},
    'large': {'S': 'https://img.scryfall.com/cards/large/front/6/1/61dc9a80-850c-4945-b84a-85381cc3ee85.jpg?1577717404'},
    'png': {'S': 'https://img.scryfall.com/cards/png/front/

In [88]:
def save_scryfall_page(table, page):
    dynamo_batches = [(first, first+BATCH_LIMIT) for first in list(range(0, CARDS_PER_PAGE, BATCH_LIMIT))]

    res = requests.get(CARDS_URL.format(page))
    for batch in dynamo_batch:
        cards = res.json()['data'][batch[0]:batch[1]]
        # Save to Dynamo
        RequestItems = {
            table: [
                {
                    'PutRequest': {
                        'Item': card
                    }
                } for card in cards
            ]
        }
        return RequestItems
        #_ = dynamodb_lib.call(table, 'batch_write_item', RequestItems)

In [113]:
items = save_scryfall_page(table, 1)

In [122]:
items['global-cards-table']

[{'PutRequest': {'Item': {'object': 'card',
    'id': '61dc9a80-850c-4945-b84a-85381cc3ee85',
    'oracle_id': 'e521322b-0e83-458c-8936-7021a80ee279',
    'multiverse_ids': [],
    'tcgplayer_id': 206636,
    'name': 'Temple of Plenty',
    'lang': 'en',
    'released_at': '2020-01-24',
    'uri': 'https://api.scryfall.com/cards/61dc9a80-850c-4945-b84a-85381cc3ee85',
    'scryfall_uri': 'https://scryfall.com/card/thb/351/temple-of-plenty?utm_source=api',
    'layout': 'normal',
    'highres_image': False,
    'image_uris': {'small': 'https://img.scryfall.com/cards/small/front/6/1/61dc9a80-850c-4945-b84a-85381cc3ee85.jpg?1577717404',
     'normal': 'https://img.scryfall.com/cards/normal/front/6/1/61dc9a80-850c-4945-b84a-85381cc3ee85.jpg?1577717404',
     'large': 'https://img.scryfall.com/cards/large/front/6/1/61dc9a80-850c-4945-b84a-85381cc3ee85.jpg?1577717404',
     'png': 'https://img.scryfall.com/cards/png/front/6/1/61dc9a80-850c-4945-b84a-85381cc3ee85.png?1577717404',
     'art_cro

In [110]:
def batch_format(val):
    if type(val) is str:
        return {"S": str(val)}
    if type(val) is int:
        return {"N": str(val)}
    if type(val) is float:
        return {"N": str(val)}
    if type(val) is list:
        return {"SS": val}
    if type(val) is bool:
        return {"BOOL": True}
    if val is None:
        return {"NULL": True}
    if type(val) is dict:
        return {"M": {val_key: batch_format(val_val) for val_key, val_val in val.items()}}

In [111]:
for key, val in items['global-cards-table'][0]['PutRequest']['Item'].items():
    items['global-cards-table'][0]['PutRequest']['Item'][key] = batch_format(val)

In [112]:
items['global-cards-table'][0]['PutRequest']['Item']

{'object': {'S': 'card'},
 'id': {'S': '61dc9a80-850c-4945-b84a-85381cc3ee85'},
 'oracle_id': {'S': 'e521322b-0e83-458c-8936-7021a80ee279'},
 'multiverse_ids': {'SS': []},
 'tcgplayer_id': {'N': '206636'},
 'name': {'S': 'Temple of Plenty'},
 'lang': {'S': 'en'},
 'released_at': {'S': '2020-01-24'},
 'uri': {'S': 'https://api.scryfall.com/cards/61dc9a80-850c-4945-b84a-85381cc3ee85'},
 'scryfall_uri': {'S': 'https://scryfall.com/card/thb/351/temple-of-plenty?utm_source=api'},
 'layout': {'S': 'normal'},
 'highres_image': {'BOOL': True},
 'image_uris': {'M': {'small': {'S': 'https://img.scryfall.com/cards/small/front/6/1/61dc9a80-850c-4945-b84a-85381cc3ee85.jpg?1577717404'},
   'normal': {'S': 'https://img.scryfall.com/cards/normal/front/6/1/61dc9a80-850c-4945-b84a-85381cc3ee85.jpg?1577717404'},
   'large': {'S': 'https://img.scryfall.com/cards/large/front/6/1/61dc9a80-850c-4945-b84a-85381cc3ee85.jpg?1577717404'},
   'png': {'S': 'https://img.scryfall.com/cards/png/front/6/1/61dc9a80-850

In [ ]:
cards = res.json()['data'][batch[0]:batch[1]]

In [4]:
res.json()['total_cards']

263830

In [5]:
import math

In [6]:
cards_per_page = 175
total_cards = res.json()['total_cards']

total_pages = math.ceil(total_cards / cards_per_page)

total_pages

1508

In [9]:
PAGES_PER_WORKER = 100

In [22]:
total_pages = 1508

In [24]:
worker_pages = [(first, first+PAGES_PER_WORKER) for first in list(range(1, total_pages+1, PAGES_PER_WORKER))]
worker_pages

[(1, 101),
 (101, 201),
 (201, 301),
 (301, 401),
 (401, 501),
 (501, 601),
 (601, 701),
 (701, 801),
 (801, 901),
 (901, 1001),
 (1001, 1101),
 (1101, 1201),
 (1201, 1301),
 (1301, 1401),
 (1401, 1501),
 (1501, 1601)]

In [15]:
CARDS_URL = 'https://api.scryfall.com/cards?page={}'

In [16]:
CARDS_URL.format(1)

'https://api.scryfall.com/cards?page=1'

In [27]:
cards_url = 'https://api.scryfall.com/cards?page=10'

In [30]:
int(str.split(cards_url, '=')[1])

10

In [33]:
import datetime

In [35]:
start = datetime.datetime.now()

In [37]:
print(datetime.datetime.now() - then)

0:00:23.186929


In [38]:
start = datetime.datetime.now()
all_cards_path = os.path.join(os.getcwd(), 'data', 'all_scryfall.json')
all_cards_df = pd.DataFrame()
cards_url = 'https://api.scryfall.com/cards?page=1'
cards_per_page = 175
has_more = True
while has_more:
#while int(str.split(cards_url, '=')[1]) < 101:
    res = requests.get(cards_url)
    
    #if int(cards_url[-1]) % 10 == 0:
    #    print('Getting page {}'.format(cards_url))
    
    cards_df = pd.DataFrame.from_dict(res.json()['data'])

    all_cards_df = pd.concat([all_cards_df, cards_df], sort=True, ignore_index=True)
          
    has_more = res.json()['has_more']
    cards_url = res.json()['next_page']
    
    sleep(0.1)

print(datetime.datetime.now() - then)
#all_cards_df.to_json(all_cards_path)

Getting page https://api.scryfall.com/cards?page=10
Getting page https://api.scryfall.com/cards?page=20
Getting page https://api.scryfall.com/cards?page=30
Getting page https://api.scryfall.com/cards?page=40
Getting page https://api.scryfall.com/cards?page=50
Getting page https://api.scryfall.com/cards?page=60
Getting page https://api.scryfall.com/cards?page=70
Getting page https://api.scryfall.com/cards?page=80
Getting page https://api.scryfall.com/cards?page=90
Getting page https://api.scryfall.com/cards?page=100
0:05:45.799994


In [8]:
all_cards_path

'C:\\Users\\LWEST\\Desktop\\mtgml-api\\notebooks\\..\\data\\all_scryfall.json'

In [9]:
all_cards_path = os.path.join(os.getcwd(), '..', 'data', 'all_scryfall.json')

all_cards_df = pd.read_json(all_cards_path)

In [10]:
all_cards_df.shape

(261438, 73)

In [13]:
all_cards_df.head(2)

,all_parts,arena_id,artist,artist_ids,booster,border_color,card_back_id,card_faces,cmc,collector_number,color_identity,color_indicator,colors,digital,edhrec_rank,flavor_text,foil,frame,frame_effects,full_art,games,hand_modifier,highres_image,id,illustration_id,image_uris,lang,layout,legalities,life_modifier,loyalty,mana_cost,mtgo_foil_id,mtgo_id,multiverse_ids,name,nonfoil,object,oracle_id,oracle_text,oversized,power,preview,prices,printed_name,printed_text,printed_type_line,prints_search_uri,promo,promo_types,purchase_uris,rarity,related_uris,released_at,reprint,reserved,rulings_uri,scryfall_set_uri,scryfall_uri,set,set_name,set_search_uri,set_type,set_uri,story_spotlight,tcgplayer_id,textless,toughness,type_line,uri,variation,variation_of,watermark
0,None,NaN,Milivoj Ćeran,[1eced451-4da5-42bc-b49d-70c41246581f],False,black,0aeebaf5-8c7d-4636-9e82-8c27447861f7,None,6.0,5,[G],None,[G],False,NaN,"""Come, my wild children. Let's give the interl...",True,2015,None,False,[paper],NaN,False,dbcdbf7a-9294-47ad-9f93-c16b78c7463a,fe97c475-ef9a-41b2-872b-2469ac9f48d7,{'small': 'https://img.scryfall.com/cards/smal...,en,normal,"{'standard': 'not_legal', 'future': 'not_legal...",NaN,None,{4}{G}{G},NaN,NaN,[],Earthshaker Giant,False,card,cd6250ae-9079-4a62-8a70-0d94fbac21bc,Trample\nWhen Earthshaker Giant enters the bat...,False,6,None,"{'usd': None, 'usd_foil': '12.00', 'eur': None...",None,None,None,https://api.scryfall.com/cards/search?order=re...,False,None,{'tcgplayer': 'https://shop.tcgplayer.com/prod...,mythic,{'tcgplayer_decks': 'https://decks.tcgplayer.c...,2019-11-15,False,False,https://api.scryfall.com/cards/dbcdbf7a-9294-4...,https://scryfall.com/sets/gn2?utm_source=api,https://scryfall.com/card/gn2/5/earthshaker-gi...,gn2,Game Night 2019,https://api.scryfall.com/cards/search?order=se...,box,https://api.scryfall.com/sets/a72727dd-dbab-41...,False,200607.0,False,6,Creature — Giant Druid,https://api.scryfall.com/cards/dbcdbf7a-9294-4...,False,None,None
1,None,NaN,Lucas Graciano,[ce98f39c-7cdd-47e6-a520-6c50443bb4c2],False,black,0aeebaf5-8c7d-4636-9e82-8c27447861f7,None,6.0,4,[R],None,[R],False,NaN,"Half the size, double the mayhem.",True,2015,None,False,[paper],NaN,False,acb3ce9b-ee4f-410a-8db3-e87aeb0a4444,56da4782-3fe9-4230-b426-8e7903fe281f,{'small': 'https://img.scryfall.com/cards/smal...,en,normal,"{'standard': 'not_legal', 'future': 'not_legal...",NaN,None,{4}{R}{R},NaN,NaN,[],Fiendish Duo,False,card,ab0dfae5-b9d4-417b-8a0d-2525ae3a73b9,First strike\nIf a source would deal damage to...,False,5,None,"{'usd': None, 'usd_foil': '15.00', 'eur': None...",None,None,None,https://api.scryfall.com/cards/search?order=re...,False,None,{'tcgplayer': 'https://shop.tcgplayer.com/prod...,mythic,{'tcgplayer_decks': 'https://decks.tcgplayer.c...,2019-11-15,False,False,https://api.scryfall.com/cards/acb3ce9b-ee4f-4...,https://scryfall.com/sets/gn2?utm_source=api,https://scryfall.com/card/gn2/4/fiendish-duo?u...,gn2,Game Night 2019,https://api.scryfall.com/cards/search?order=se...,box,https://api.scryfall.com/sets/a72727dd-dbab-41...,False,200606.0,False,5,Creature — Devil,https://api.scryfall.com/cards/acb3ce9b-ee4f-4...,False,None,None


In [81]:
url = all_cards_df.sample(5).image_uris.values[4]['normal']

In [82]:
from PIL import Image
import requests
from io import BytesIO

response = requests.get(url)
img = Image.open(BytesIO(response.content))
img.show()